In [ ]:
import os
from Bio import SeqIO
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from ete3 import Tree
from scipy.spatial.distance import pdist, squareform
from time import time

In [ ]:
# Step 1: Reading FASTA Files
data_folder = './data/Fish'
fasta_files = [f for f in os.listdir(data_folder) if f.endswith('.fasta')]

In [ ]:
# Extracting Accession Numbers and Reading Arrays
sequences = []
titles = []
for file in fasta_files:
    filepath = os.path.join(data_folder, file)
    for record in SeqIO.parse(filepath, 'fasta'):
        sequences.append(str(record.seq))
        titles.append(record.id) 

In [ ]:
# Step 2: Function to Obtain n-grams
def extract_ngrams(sequence, n):
    return [sequence[i:i+n] for i in range(len(sequence)-n+1)]

In [ ]:
# Reading Reference Dendrogram
with open(os.path.join(data_folder,'reference_newick.txt'), 'r') as file:
    reference_newick = file.read().strip()
reference_tree = Tree(reference_newick, format=1)

In [ ]:
# n-gram sizes and TF-IDF percentiles
n_gram_sizes = list(range(3, 21))
percentile_ranges = list(range(1, 21)) + list(range(30, 101, 10))

In [ ]:
results = []
newick_trees = []

In [ ]:
for n in n_gram_sizes:
    for percentile in percentile_ranges:
        start_time = time()
        
        all_ngrams = [extract_ngrams(seq, n) for seq in sequences]
        top_ngrams_per_sequence = []

        for ngrams in all_ngrams:
            ngram_text = ' '.join(ngrams)
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform([ngram_text])
            tfidf_scores = pd.DataFrame(tfidf_matrix.T.toarray(), index=vectorizer.get_feature_names_out(), columns=['TF-IDF'])
            
            # Select n-grams by percentile
            num_top_ngrams = int(np.ceil(len(tfidf_scores) * (percentile / 100)))
            top_ngrams = tfidf_scores.nlargest(num_top_ngrams, 'TF-IDF')
            top_ngrams_per_sequence.append(top_ngrams.index.tolist())
        
        # Creating Similarity Matrix
        num_sequences = len(sequences)
        similarity_matrix = np.zeros((num_sequences, num_sequences))
        
        for i in range(num_sequences):
            for j in range(num_sequences):
                if i != j:
                    common_ngrams = len(set(top_ngrams_per_sequence[i]).intersection(set(top_ngrams_per_sequence[j])))
                    similarity_matrix[i, j] = common_ngrams
        
        # Convert similarity matrix to distance matrix
        max_similarity = similarity_matrix.max()
        if max_similarity == 0:
            max_similarity = 1 

        distance_matrix = 1 - similarity_matrix
        min_val = distance_matrix.min()
        max_val = distance_matrix.max()
        normalized_distance_matrix = (distance_matrix - min_val) / (max_val - min_val)

        # Convert square similarity matrix to condensed distance vector
        condensed_distance_matrix = squareform(normalized_distance_matrix, checks=False)
        
        # Creating Dendrogram
        linked = sch.linkage(condensed_distance_matrix, method='average')
        labels = titles
        
       # function to convert scipy linkage matrix to newick format
        def linkage_to_newick(linkage, labels):
            tree = sch.to_tree(linkage, rd=False)
            def build_newick(node, newick, leaf_names):
                if node.is_leaf():
                    return "%s%s" % (leaf_names[node.id], newick)
                else:
                    if len(newick) > 0:
                        newick = ")%s" % (newick)
                    else:
                        newick = ");"
                    newick = build_newick(node.get_left(), newick, leaf_names)
                    newick = build_newick(node.get_right(), ",%s" % newick, leaf_names)
                    newick = "(%s" % newick
                    return newick
            return build_newick(tree, "", labels)

        tree_newick = linkage_to_newick(linked, labels)
        nwck = [n,"-",percentile,"-",tree_newick]
        newick_trees.append(nwck)
        # nRF calculation using ETE3
        generated_tree = Tree(tree_newick, format=1)
        nrf = reference_tree.compare(generated_tree, unrooted=True)['norm_rf']

        # Accuracy calculation
        accuracy = (1 - nrf) * 100
        
        end_time = time()
        computation_time = end_time - start_time
        
        print({
            'n-gram Size': n,
            'Percentile': percentile,
            'Accuracy (%)': accuracy,
            'Computation Time (s)': computation_time
        })

        results.append({
            'n-gram Size': n,
            'Percentile': percentile,
            'Accuracy (%)': accuracy,
            'Computation Time (s)': computation_time
        })
       

In [ ]:
df=pd.DataFrame(results)
pd.set_option('display.max_rows', None)

In [ ]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by=['Accuracy (%)','n-gram Size','Percentile'], ascending=False)